In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
import numpy as np

# ==========================
# 1. Criar dataset fictício
# ==========================
data = {
    'amount': [100, 110, 90, 95, 105, 120, 80, 130, 125, 115],
    'country': ['BR', 'BR', 'BR', 'AR', 'AR', 'AR', 'BR', 'AR', 'BR', 'AR'],
    'sales_person': ['Ana', 'Ana', 'Carlos', 'Carlos', 'Luiz', 'Luiz', 'Ana', 'Carlos', 'Luiz', 'Ana'],
    'date': pd.date_range(start='2023-01-01', periods=10, freq='D'),
    'ticket': [500, 520, 480, 470, 510, 530, 450, 550, 540, 500],
    'product': ['A', 'A', 'B', 'B', 'C', 'C', 'A', 'B', 'C', 'A'],
    'boxes_shipped': [50, 48, 60, 58, 55, 53, 65, 57, 52, 49]
}

df = pd.DataFrame(data)

# ==========================
# 2. Pré-processamento simples
# ==========================
# Converter categóricas para dummies
df_encoded = pd.get_dummies(df.drop(columns=['date']), drop_first=True)

# ==========================
# 3. Treinar modelo
# ==========================
X = df_encoded.drop(columns=['boxes_shipped'])
y = df_encoded['boxes_shipped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

print("R² no teste:", round(r2_score(y_test, model.predict(X_test)), 3))

# ==========================
# 4. Função para simular aumento e calcular elasticidade
# ==========================
def calcular_elasticidade(produto, aumento_percentual):
    # Filtrar produto
    base_prod = df[df['product'] == produto].copy()

    # Criar cópias para cenário original e com aumento
    df_original = base_prod.copy()
    df_aumento = base_prod.copy()
    df_aumento['amount'] *= (1 + aumento_percentual / 100)

    # Codificar igual ao treino
    X_original = pd.get_dummies(df_original.drop(columns=['date', 'boxes_shipped']), drop_first=True)
    X_aumento = pd.get_dummies(df_aumento.drop(columns=['date', 'boxes_shipped']), drop_first=True)

    # Garantir mesmas colunas
    X_original = X_original.reindex(columns=X.columns, fill_value=0)
    X_aumento = X_aumento.reindex(columns=X.columns, fill_value=0)

    # Prever
    pred_original = model.predict(X_original).mean()
    pred_aumento = model.predict(X_aumento).mean()

    # Calcular elasticidade
    delta_q = (pred_aumento - pred_original) / pred_original
    delta_p = aumento_percentual / 100
    elasticidade = delta_q / delta_p

    return {
        'produto': produto,
        'vendas_originais': pred_original,
        'vendas_com_aumento': pred_aumento,
        'elasticidade': elasticidade
    }

# ==========================
# 5. Exemplo de uso
# ==========================
resultado = calcular_elasticidade(produto='A', aumento_percentual=50)
print(resultado)


R² no teste: -6.103
{'produto': 'A', 'vendas_originais': 54.873893388665216, 'vendas_com_aumento': 52.825358174034754, 'elasticidade': -0.07466338136865676}
